In [1]:
import torch 
import sys
sys.path.insert(0, 'src/')

from src.model.vae import VAE_Encoder, VAE_Decoder
from src.model.config import StableDiffusionConfig
from src.model.unet import UNet

from src.model.clip import CLIPEncoder

config = StableDiffusionConfig()

VAE

In [2]:
enc = VAE_Encoder(config.img_channels, config.vae_features_dims, config.vae_num_groups, config.vae_num_heads, config.vae_dropout, config.vae_latent_dim)

print(sum(p.numel() for p in enc.parameters()))

23237584


In [3]:
img = torch.randn((1, config.img_channels, config.img_size, config.img_size))
# noise = torch.randn((config.batch_size, config.vae_latent_dim // 2, config.img_size // 2**(len(config.vae_features_dims)-1), config.img_size // 2**(len(config.vae_features_dims)-1)))
noise = torch.randn((1, 4, 64, 64)).to('cuda')
enc = enc.to('cuda')
out = enc(img.to('cuda'), noise)

torch.Size([1, 4, 64, 64]) torch.Size([1, 4, 64, 64])


UNET

In [4]:
model = UNet(config.vae_latent_dim, config.unet_features_dims, config.unet_attn_num_heads, config.unet_attn_dim, config.unet_time_emb_dim, config.unet_time_emb_dim_scale_factor)

In [5]:
sum(p.numel() for p in model.parameters())

852620804

In [6]:
time = torch.randn((1, config.unet_time_emb_dim))
model(out, time)

TypeError: SwitchSequential.forward() missing 1 required positional argument: 'time'

In [2]:
tokens = torch.randint(low=0, high=config.vocab_size, size=(1, config.clip_seq_len)).to('cuda')
tokens.shape

torch.Size([1, 77])

In [3]:
clip = CLIPEncoder(config.vocab_size, config.clip_emb_dim, config.clip_seq_len,
                   config.clip_attn_num_heads, config.clip_emb_dim_scale_factor, 
                   config.clip_num_layers, config.clip_dropout).to('cuda')

In [4]:
sum(p.numel() for p in clip.parameters())

123060480

In [ ]:
UNet_AttentionBlock

In [6]:
text_emb = clip(tokens)

In [7]:
text_emb.shape

torch.Size([1, 77, 768])

In [4]:
from torch import nn

In [5]:
out.shape

torch.Size([1, 4, 64, 64])

In [6]:
dec = VAE_Decoder(config.vae_latent_dim, config.vae_features_dims, config.vae_num_groups, config.vae_dropout, config.vae_num_heads, config.img_channels)

In [7]:
print(sum(p.numel() for p in dec.parameters()))

32367379


In [8]:
dec = dec.to('cuda')

In [9]:
dec(out).shape

torch.Size([1, 4, 64, 64])
torch.Size([1, 512, 64, 64])
torch.Size([1, 512, 64, 64])
torch.Size([1, 512, 128, 128])
torch.Size([1, 256, 256, 256])
torch.Size([1, 128, 512, 512])


torch.Size([1, 3, 512, 512])